In [1]:
import bcolz
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
inputs = bcolz.open('/data/poker/predict_action_input_2.dat')
outputs = bcolz.open('/data/poker/predict_action_output_2.dat')

cards_data = np.array(inputs[:,0].tolist())
my_bet_data = np.array(inputs[:,1].tolist())
bets_data = np.array(inputs[:,2].tolist()).reshape((inputs.shape[0], 3, 6,1))

In [4]:
from keras.layers import Input, Dense, Flatten, concatenate, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv2D, ZeroPadding2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
import keras.layers
from keras.optimizers import Nadam

cards_input = Input(shape=(4, 13, 4))
my_bet_input = Input(shape=(3, ))
bets_input = Input(shape=(3,6,1))

cx = ZeroPadding2D(padding=(0, 2))(cards_input)
cx = Conv2D(100,(4,5), padding='same', activation='relu')(cx)
cx = Conv2D(50,(4,5), padding='valid', activation='relu')(cx)
cx = Flatten()(cx)

bx = Conv2D(50,(3,1), padding='valid', activation='relu')(bets_input)
bx = Conv2D(25,(1,1), padding='valid', activation='relu')(bx)
bx = Flatten()(bx)

tx = concatenate([cx, my_bet_input, bx])
tx = Dense(100, activation='relu')(tx)
tx = Dense(50, activation='relu')(tx)
bet_output = Dense(4, name='bet_output', activation='softmax')(tx)

model = Model([cards_input, my_bet_input, bets_input], [bet_output])

opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.006)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.fit([cards_data, my_bet_data, bets_data], outputs, shuffle=True, validation_split=0.05, epochs=5)

/opt/conda/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:2252: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 427710 samples, validate on 22512 samples
Epoch 1/5
427710/427710 [==============================] - 126s - loss: 0.5962 - acc: 0.7638 - val_loss: 0.4731 - val_acc: 0.8120
Epoch 2/5
427710/427710 [==============================] - 127s - loss: 0.4873 - acc: 0.8085 - val_loss: 0.4567 - val_acc: 0.8171
Epoch 3/5
427710/427710 [==============================] - 126s - loss: 0.4635 - acc: 0.8178 - val_loss: 0.4221 - val_acc: 0.8323
Epoch 4/5
427710/427710 [==============================] - 127s - loss: 0.4499 - acc: 0.8231 - val_loss: 0.4388 - val_acc: 0.8279
Epoch 5/5
427710/427710 [==============================] - 126s - loss: 0.4403 - acc: 0.8272 - val_loss: 0.4261 - val_acc: 0.8316


In [7]:
model.save_weights('/data/trained_models/poker/predict_action_v4.h5')

In [5]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 4, 13, 4)      0                                            
____________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D) (None, 4, 17, 4)      0           input_4[0][0]                    
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 3, 6, 1)       0                                            
____________________________________________________________________________________________________
conv2d_5 (Conv2D)                (None, 4, 17, 100)    8100        zero_padding2d_2[0][0]           
___________________________________________________________________________________________